In [13]:
import pandas as pd
import mysql.connector as mysql
import numpy as np

# Raw Data Loading in DataFrame, Cleaning and initializing

In [12]:
#Name of each columns
column_name=["Customer Name","Customer ID","Customer Open Date","Last Consulted Date","Vaccination Type",
             "Doctor Consulted","State","Country","Date of Birth","Active Customer"]


# Read pipe delimited data file
# sep - the separator of data "|"
# skiprows - it skips rows from beginning.1 used to skip header row.
# skipfooter - it skips rows from ending.0 used because there is not any footer row in the end. if footer is avaible then use 1.
# usecols - list of index of which columns needs to be added in dataframe. here first two columns ignored because first was empty and second defines |D|.
# names - set column names. gave list of column name respectively.
# parse_dates - used to convert date-time data from string to date time object. gave list of columns number
data=pd.read_csv("data.csv",sep="|",skiprows=1,skipfooter=0,usecols=[2,3,4,5,6,7,8,9,10,11],names=column_name,parse_dates=[2,3,8])


# it is important converted all datatype to string for using it in insert query of table which is dynamically formed.
data=data.astype(str)

#replace empty or nan in non-mandatory Date fields to the default 00000000 which will be interpreted as None in date type object of sql 
data["Last Consulted Date"]=data["Last Consulted Date"].replace("nan","00000000")
data["Date of Birth"]=data["Date of Birth"].replace("nan","00000000")

#coverting "nan" string to None object.
data[data.loc[:]=="nan"]=None

# "Customer Name","Customer ID","Customer Open Date" are mandatory fields 
# so if any records have missing values in that columns then 
# we have to remove that data or we can store somewhere else.
# here we extracted indexes of data where mandatory field containing "None" or "NaT" 
bad_data_indx=np.concatenate((np.where(data[["Customer Name","Customer ID","Customer Open Date"]]=="NaT")[0],
                              np.where(data[["Customer Name","Customer ID","Customer Open Date"]].isna())[0]))

# storing bad-data into pipe delimited data file so we can recover missing data later
data.loc[bad_data_indx,:].to_csv("bad_data.csv",sep="|")

#removing bad-data from our main data so we can procceed it further.
data.drop(index=bad_data_indx,inplace=True)


data

,Customer Name,Customer ID,Customer Open Date,Last Consulted Date,Vaccination Type,Doctor Consulted,State,Country,Date of Birth,Active Customer
0,Alex,123457,2010-10-12,2012-10-13,MVD,Paul,SA,USA,00000000,A
1,John,123458,2010-10-12,2012-10-13,MVD,Paul,None,IND,06031987,A
2,Mathew,123459,2010-10-12,2012-10-13,MVD,Paul,WAS,PHIL,06031987,A
3,Matt,12345,2010-10-12,2012-10-13,MVD,Paul,BOS,NYC,06031987,A
4,Jacob,1256,2010-10-12,2012-10-13,MVD,Paul,VIC,AU,06031987,A
5,Jack,12956,2010-10-12,2012-10-13,None,Paul,None,AU,06031987,A
8,meet,10256,2010-10-12,2012-10-13,MVD,Paul,None,AU,06031987,A


# Connect to Mysql Database

In [3]:
user_name="root"
password="12345"
db_name="incubyte"
try:
    connection = mysql.connect(host='localhost',
                                         database=db_name,
                                         user=user_name,
                                         password=password)
    if connection.is_connected():
        cursor = connection.cursor(buffered=True)
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
except Exception as e:
    print("Error while connecting to MySQL", e)


You're connected to database:  ('incubyte',)


# Create Table 

In [4]:
def create_table(name,columns_name,column_spec):
    cursor=connection.cursor()
    cursor.execute("SHOW TABLES FROM "+db_name)
    tables=[i[0] for i in cursor.fetchall()]
    if (name not in tables) and (len(columns_name)==len(column_spec)):
        try:
            tbl_cols=[(" ").join([i,j]) for i,j in zip(columns_name,column_spec)]
            query='CREATE TABLE  IF NOT EXISTS '+name+' (' + ', '.join(tbl_cols) + ');'
            cursor.execute(query)
            print(name+" table successfully created")
        except Exception as e:
            print("Error while creating table: "+e)
    else:
        if (len(columns_name)!=len(column_spec)):
            print("number of Column names and column specifications not same.")
        else:
            print("Table already exists")
    cursor.close()

# Insert  Records in Table

In [5]:
#list of tuples for each records
def insert_records(table_nm,records):
    try:
        cursor=connection.cursor()
        qry="insert into "+table_nm+" VALUES(%s, %s ,%s ,%s ,%s ,%s ,%s ,%s , STR_TO_DATE(%s, '%d%m%Y') ,%s);"
        cursor.executemany(qry,records)
        cursor.close()
    except Exception as e:
            print("Error while insert records : "+str(e))
    


# Show Mysql Table data in DataFrame

In [6]:
def Show_Table(table_name):
    cursor=connection.cursor()
    cursor.execute("select * from "+table_name)
    read_data=pd.DataFrame(cursor.fetchall())
    cursor.close()
    return read_data

# Common Table Fields 

In [7]:
#Created list of table details as per mentioned in instruction
columns_name=["Customer_Name","Customer_ID","Customer_Open_Date","Last_Consulted_Date","Vaccination_Type",
             "Doctor_Consulted","State","Country","Date_of_Birth","Active_Customer"]

column_spec=["VARCHAR(255) NOT NULL PRIMARY KEY","VARCHAR(18) NOT NULL","DATE NOT NULL","DATE","VARCHAR(5)",
             "VARCHAR(255)","VARCHAR(5)","VARCHAR(5)","DATE","VARCHAR(1)"]

# Fianally Load Clean Data to Database

In [8]:
#list of unique country to create table accordingly
countries=data["Country"].unique()

#country-wise  data slicing and storing to corresponding table
for country in countries: 
    #create_table if not exist
    create_table(country,columns_name,column_spec)
    
    #extract data of particular country
    country_data=data[data["Country"]==country]
    #convert to list of records from dataframe
    records=list(country_data.to_numpy())
    
    #Insert data into corresponding table
    insert_records(country,records)

USA table successfully created
IND table successfully created
PHIL table successfully created
NYC table successfully created
AU table successfully created


# List of Tables

In [9]:
cursor=connection.cursor()
cursor.execute("SHOW TABLES FROM "+db_name)
tables=[i[0] for i in cursor.fetchall()]
cursor.close()
print("Tables from Database",tables)

Tables from Database ['au', 'ind', 'nyc', 'phil', 'usa']


# Read the Data from Database Tables

In [10]:
table_name="au"
Show_Table(table_name)

,0,1,2,3,4,5,6,7,8,9
0,Jack,12956,2010-10-12,2012-10-13,None,Paul,None,AU,1987-03-06,A
1,Jacob,1256,2010-10-12,2012-10-13,MVD,Paul,VIC,AU,1987-03-06,A
2,meet,10256,2010-10-12,2012-10-13,MVD,Paul,None,AU,1987-03-06,A
